# `%rites` and `%%rites`



In [77]:
    from contextlib import contextmanager

In [114]:
    alias = {str: f"rites.{str}" for str in ('markdown', 'template', 'conventions')}
    alias['test'] = 'sweet.test'
    alias['typing'] = 'sweet.typing'

In [111]:
    from argparse import ArgumentParser

    parser = ArgumentParser('rites', fromfile_prefix_chars='@')
    parser.add_argument(
        'on', nargs='*', 
        default=[],
        help='Turn on IPython magics by package path.'
    )
    parser.add_argument(
        '-🚫', '--off', nargs='*', default=[],
        help='Turn off IPython magics package path or as a file.'
    );
    def rites(line, cell=None):
        ip = __import__('IPython').get_ipython()

        def magics_state(on=[], off=[]):
            from importlib import import_module
            from warnings import warn
            global alias
            nonlocal ip
            for object, prefix in zip((on, off), ('', 'un')):
                for object in [alias.get(str, str) for str in object]:
                    try:
                        getattr(import_module(object), f'{prefix}load_ipython_extension')(ip)
                    except AttributeError as e:
                        warnings.warn(str(e))
                        
        object = parser.parse_args(line.split())
        
        magics_state(object.on, object.off)
        
        if cell:
            try: ip.run_cell(cell)
            except: ...
            finally: magics_state([], object.on)
    



In [112]:
    def load_ipython_extension(ip=None):
        ip = ip or __import__('IPython').get_ipython()
        ip.register_magic_function(rites, 'line_cell')
        
    def unload_ipython_extension(ip=None):
        del (ip or __import__('IPython').get_ipython()).magics_manager.magics['line']['rites']

In [108]:
    def test():
        def areload():
            try: del ip.magics_manager.magics['line']['autoreload']
            except: ...
        
        ip = __import__('IPython').get_ipython()
        load_ipython_extension()
        areload()
        assert 'autoreload' not in ip.magics_manager.magics['line']
        %rites autoreload
        assert 'autoreload' in ip.magics_manager.magics['line']
        assert 'rites' in ip.magics_manager.magics['line']
        %rites --off __main__
        assert 'rites' not in ip.magics_manager.magics['line']
        areload()
        
    def test_file(): ...

In [109]:
    if __name__ == '__main__':
        from unittest import FunctionTestCase, TestResult
        result = TestResult()
        [FunctionTestCase(object).run(result) for object in (test, test_file)]
        print(result)
        load_ipython_extension()

<unittest.result.TestResult run=2 errors=0 failures=0>
